# Chatbot

## Install Dependencies

In [ ]:
# pip install -qU langchain langchain-core langgraph

## Setup LangSmith Tracing

In [1]:
import os
import getpass

if "LANGSMITH_API_KEY" not in os.environ:
    os.environ["LANGSMITH_TRACING"] = "true"
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter LangSmith API key: ")

if "GEMINI_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter Google API key: ")

## Initialize Chat Model

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google-genai")

c:\Users\Dede\Downloads\nextjs-langchain-chatbot\prototype\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Create Prompt Templates

In [13]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_templates = ChatPromptTemplate.from_messages([
    ("system", "You talk like a jarvis. Answer all questions to the best of your ability."),
    MessagesPlaceholder(variable_name="messages")
])

## Limit Messages Size

In [14]:
from langchain_core.messages import trim_messages

trimmer = trim_messages(
    max_tokens=81920,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

## Define and Compile Workflow

In [15]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define the graph
workflow = StateGraph(state_schema=MessagesState)

# Define the function that calls the model
def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_templates.invoke(
        {"messages": trimmed_messages   }
    )
    response = model.invoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

## Create a Config

In [16]:
config = {"configurable": {"thread_id": "test123"}}

## Invoke the App

In [ ]:
from langchain_core.messages import HumanMessage

query = "Hi! I'm Iron Man"

input_messages = [HumanMessage(query)]
for chunk, metadata in app.stream(
    {"messages": input_messages},
    config,
    stream_mode="messages"
):
    if isinstance(chunk, HumanMessage):
        print(chunk.content, end="|")